In [ ]:
from pdf2image import convert_from_path

# Path to the PDF file

pdf_path = './file.pdf'

# Convert PDF to images
images = convert_from_path(pdf_path)

# Save each page as an image
for i, image in enumerate(images):
    image.save(f'page_{i + 1}.png', 'PNG')

In [ ]:
from IPython.display import display, Image, Audio
import cv2  
import base64
import time
from openai import OpenAI
import os
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    
)

client.temperature = 0.5
client.seed=42

deployment_name=os.getenv("GPT4O_MODEL_DEPLOYMENT_NAME")


In [ ]:
import os
import base64
from PIL import Image
from io import BytesIO
import re

# Function to extract the numeric part of the filename
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else 0

# Function to encode image to base64
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Get list of image files in the current directory
image_files = sorted([f for f in os.listdir('.') if f.endswith(('.png', '.jpg', '.jpeg'))], key=extract_number)
print(image_files)
#image_files = image_files[:5]  # Limit to first 5 images

# Encode images to base64
base64_images = [encode_image_to_base64(image) for image in image_files]



In [ ]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
system_prompt_template = env.get_template('system_prompt_template.j2')
system_prompt = system_prompt_template.render()
user_prompt_template = env.get_template('user_prompt_template.j2')
user_prompt = user_prompt_template.render()

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": [
        {"type": "text", "text":user_prompt},
        *map(lambda x: {"type": "image_url", 
                        "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "high"}}, base64_images),
        ],
    }
],
    temperature=0,
)
print(response.choices[0].message.content)